In [ ]:
import numpy as np
import pandas as pd
import os
from datetime import datetime
import matplotlib.pyplot as plt
from matplotlib import patches
from scipy.spatial.transform import Rotation as R
import gtsam

In [ ]:
interp_hertz = 200

def read_optitrack_csv(filename, start_time_string):
    dt =datetime.strptime(start_time_string, '%Y-%m-%d %I.%M.%S.%f%p')
    timestamp = datetime.timestamp(dt)
    df = pd.read_csv(filename, header=[5])
    df = df.dropna()
    df['Time (Seconds)'] += timestamp
    df = df.rename(columns={"Time (Seconds)" : "timestamp", "X.1": "tx", "Y.1" : "ty", "Z.1" : "tz"})
    return df

optitrack = read_optitrack_csv("/data/05_14_2022/inside_loop_opti.csv","2022-05-14 03.43.29.875PM" )
print(optitrack.head())

In [ ]:
#interpolate missing readings in optitrack - Not needed now
gt_data = optitrack.drop(['Frame', "Unnamed: 9"], axis=1)
cols=['timestamp', 'tx', 'ty', 'tz', 'X', 'Y', 'Z', 'W']
gt_data = gt_data[cols]
print(gt_data.index)
#gt_data = gt_data[gt_data.index % 20 == 0]
theta = np.pi
rot_apply = np.array([[np.cos(theta), np.sin(theta), 0],[-1*np.sin(theta), np.cos(theta), 0],[0,0,1]])
print(rot_apply)
for row in range(gt_data.shape[0]):
    quat = np.array(gt_data.iloc[row, 4:8])
    rot = R.from_quat(quat).as_matrix()
    rot_1 =  rot_apply @ rot
    #rint(gt_data.iloc[row])
    gt_data.iloc[row, 4:8] = R.from_matrix(rot_1).as_quat()
    gt_data.iloc[row, 1:4] = rot_apply @ gt_data.iloc[row, 1:4]
    #rint(gt_data.iloc[row])
    #print(rot)
    #print(rot_1)
    #print("---------")
#If we need to transform the data we can - Not needed for now
gt_data.to_csv("/data/05_14_2022/inside_loop/results/inside_loop_opti.txt", sep=' ', header=False, index=False)

In [ ]:
from scipy.spatial.transform import Rotation as R
def undo_rot(quat):
    rot = R.from_quat(quat).as_matrix()
    rot_undone = rot @ (np.array([[0, 0, -1], [1, 0, 0], [0, -1, 0]]).T)
    r = R.from_matrix(rot_undone)
    return r.as_quat()
    
df = pd.read_csv("28_08_2020_trajectories/Tum_trajectrory.txt", header=None, sep=' ')
print(df.head())
df2 = df.apply(lambda row: undo_rot(row.values[-4:]), axis=1)
for i in range(df.shape[0]):
    df.iloc[i, -4:] = df2[i] 
df.head()
df.to_csv("28_08_2020_trajectories/Tum_trajectrory_conv.txt",sep=' ', header=False, index=False)

In [ ]:
trans_opti_first = np.array([[ 2.49348664284], [-2.04849290848], [1.07804751396]])  
R_opti_first = R.from_quat([0.00842613261193, 0.0183087382466, -0.0918392241001, 0.995569944382]).as_matrix()
T_opti_first = np.vstack((np.hstack((R_opti_first, trans_opti_first)) , np.array([[0,0,0,1]])))
print(T_opti_first)
df = pd.read_csv("28_08_2020_trajectories/Tum_trajectrory_conv.txt", header=None, sep=' ')
for i in range(df.shape[0]):
    rot = R.from_quat(df.iloc[i, -4:]).as_matrix()
    trans = np.array([df.iloc[i, 1: 4]]).T
    T_first_rig = np.eye(4)
    T_first_rig[:3, :3] = rot
    T_first_rig[:3, 3:4]= trans
    T_opti_rig = T_opti_first @ T_first_rig
    df.iloc[i, -4:] = R.from_matrix(T_opti_rig[:3,:3]).as_quat()
    df.iloc[i, 1:4] = T_opti_rig[:3, 3:].T
df.to_csv("28_08_2020_trajectories/Tum_trajectrory_conv.txt",sep=' ', header=False, index=False)
#print(df.iloc[0])
#rot = R.from_quat(df.iloc[0, -4:]).as_matrix()
#trans = np.array([df.iloc[0, 1: 4]]).T
#T_first_rig = np.eye(4)
#T_first_rig[:3, :3] = rot
#T_first_rig[:3, 3:4]= trans
#T_opti_rig = T_opti_first @ T_first_rig
#print(R.from_matrix(T_opti_rig[:3,:3]).as_quat())
#print(T_opti_rig[:3, 3:].T)
#df.iloc[0, -4:] = R.from_matrix(T_opti_rig[:3,:3]).as_quat()
#df.iloc[0, 1:4] = T_opti_rig[:3, 3:].T
#df.iloc[0, :3].shape
#print(df.iloc[0])

In [ ]:
import numpy as np
### sidecam2 or right side cam
T_ic5 = np.array([[-0.00927631, -0.0037559,  -0.99994992, -0.05856117],
                 [-0.99995663,  0.00086417,  0.00927313,  0.052293  ],
                 [ 0.00082929,  0.99999257, -0.00376376, -0.00098621],
                 [ 0. ,         0. ,         0. ,         1.        ]]) 

#sidecam1 or leftside cam
T_ic6 = np.array([[-0.0072078,  -0.00258906,  0.99997067,  0.06797633],
                 [ 0.99996924,  0.00307407,  0.00721575, -0.01817503],
                 [-0.00309266,  0.99999192,  0.00256682, -0.00964824],
                 [ 0. ,         0. ,         0. ,         1.        ]])

T_ic2 = np.array([[-0.99937297,  0.00280304,  0.03529594,  0.00024192],
                 [ 0.03530387,  0.00283168,  0.99937261,  0.02224647],
                 [ 0.00270134,  0.99999206, -0.00292886,  0.06674015],
                 [ 0. ,         0. ,         0. ,         1.        ]])
central, left, right

T_c6_c2 = np.linalg.inv(T_ic6) @ T_ic2
T_c5_c6 = np.linalg.inv(T_ic5) @ T_ic6

print(T_c6_c2)
print(T_c5_c6)

In [ ]:
df = pd.read_csv("ORB_stereo_noloop.txt", header=None, sep=' ')
df.head()
df[0] = df[0]* 1e-9
df.to_csv("ORB_stereo_noloop.txt",sep=' ', header=False, index=False)

In [ ]:
from scipy.spatial.transform import Rotation as R
def undo_rot(quat):
    rot = R.from_quat(quat).as_matrix()
    rot_undone = rot @ (np.array([[-1, 0, 0], [0, 0, 1], [0, 1, 0]]))
    r = R.from_matrix(rot_undone)
    return r.as_quat()    
df = pd.read_csv("/home/auv/ros_ws/src/light-fields-pack/scripts/python/optitrackGroundtruth.txt", header=None, sep=' ')
print(df.head())
df2 = df.apply(lambda row: undo_rot(row.values[-4:]), axis=1)
for i in range(df.shape[0]):
    df.iloc[i, -4:] = df2[i]
    #df.iloc[i, 1:4] = (np.array([[-1, 0, 0], [0, 0, 1], [0, 1, 0]])) @ df.iloc[i, 1:4]
df.head()
df.to_csv("/home/auv/ros_ws/src/light-fields-pack/scripts/python/optitrackGroundtruth_1.txt",sep=' ', header=False, index=False)

In [ ]:
l = np.array([[-5.456451595755009,-0.08204201901518751, -34.18691026193634]])
x_wc = np.array([[-0.3130189548504393, -0.02890853794180069, 0.9493068157020713, -4.446974418096292],
 [0.00211373344198866, 0.9995129720623712, 0.03113439914278101, -0.07702016144091782],
 [-0.9497445267203573, 0.01175223864262696, -0.3128054009310132, -34.52222460694547],
 [0, 0, 0, 1]])
x_cw = np.linalg.inv(x_wc)
l_c = x_cw @ np.vstack((l.T,[1]))
print(l)
print(l_c)

In [ ]:
##############################################################
#PLOT TRACKING STATS
##############################################################
from mpl_toolkits.mplot3d import Axes3D
from typing import Iterable, Optional, Tuple
from functools import reduce
import numpy as np
import pandas as pd
import os
from datetime import datetime
import matplotlib.pyplot as plt
from matplotlib import patches
from scipy.spatial.transform import Rotation as R

# for creating a responsive plot
%matplotlib widget

def ellipsoid(rx: float, ry: float, rz: float,
              n: int) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    Numpy equivalent of Matlab's ellipsoid function.

    Args:
        rx: Radius of ellipsoid in X-axis.
        ry: Radius of ellipsoid in Y-axis.
        rz: Radius of ellipsoid in Z-axis.
        n: The granularity of the ellipsoid plotted.

    Returns:
        The points in the x, y and z axes to use for the surface plot.
    """
    u = np.linspace(0, 2 * np.pi, n + 1)
    v = np.linspace(0, np.pi, n + 1)
    x = -rx * np.outer(np.cos(u), np.sin(v)).T
    y = -ry * np.outer(np.sin(u), np.sin(v)).T
    z = -rz * np.outer(np.ones_like(u), np.cos(v)).T

    return x, y, z


def plot_pose3_on_axes(axes, pose, color, axis_length=0.1, P=None, scale=1):
    """
    Plot a 3D pose on given axis `axes` with given `axis_length`.

    Args:
        axes (matplotlib.axes.Axes): Matplotlib axes.
        point (gtsam.Point3): The point to be plotted.
        linespec (string): String representing formatting options for Matplotlib.
        P (numpy.ndarray): Marginal covariance matrix to plot the uncertainty of the estimation.
    """
    # get rotation and translation (center)
    gRp = pose[0:3, 0:3]  # rotation from pose to global
    origin = pose[0:3,3]

    # draw the camera axes
    x_axis = origin + gRp[:, 0] * axis_length
    line = np.append(origin[np.newaxis], x_axis[np.newaxis], axis=0)
    axes.plot(line[:, 0], line[:, 1], line[:, 2], 'r-')

    y_axis = origin + gRp[:, 1] * axis_length
    line = np.append(origin[np.newaxis], y_axis[np.newaxis], axis=0)
    axes.plot(line[:, 0], line[:, 1], line[:, 2], 'g-')

    z_axis = origin + gRp[:, 2] * axis_length
    line = np.append(origin[np.newaxis], z_axis[np.newaxis], axis=0)
    axes.plot(line[:, 0], line[:, 1], line[:, 2], 'b-')

    # plot the covariance
    if P is not None:
        # covariance matrix in pose coordinate frame
        # convert the covariance matrix to global coordinate frame
        gPp = gRp @ P @ gRp.T
        plot_covariance_ellipse_3d(axes, origin, gPp, color)

def plot_covariance_ellipse_3d(axes,
                               origin: np.ndarray,
                               P: np.ndarray,
                               color:np.ndarray,
                               scale: float = 1,
                               n: int = 8,
                               alpha: float = 0.1) -> None:
    """
    Plots a Gaussian as an uncertainty ellipse

    Based on Maybeck Vol 1, page 366 
    k=2.296 corresponds to 1 std, 68.26% of all probability
    k=11.82 corresponds to 3 std, 99.74% of all probability

    Args:
        axes (matplotlib.axes.Axes): Matplotlib axes.
        origin: The origin in the world frame.
        P: The marginal covariance matrix of the 3D point
            which will be represented as an ellipse.
        scale: Scaling factor of the radii of the covariance ellipse.
        n: Defines the granularity of the ellipse. Higher values indicate finer ellipses.
        alpha: Transparency value for the plotted surface in the range [0, 1].
    """
    k = 11.82
    K= 1.0
    U, S, _ = np.linalg.svd(P)

    radii = k * np.sqrt(S)
    radii = radii * scale
    rx, ry, rz = radii

    # generate data for "unrotated" ellipsoid
    xc, yc, zc = ellipsoid(rx, ry, rz, n)

    # rotate data with orientation matrix U and center c
    data = np.kron(U[:, 0:1], xc) + np.kron(U[:, 1:2], yc) + \
        np.kron(U[:, 2:3], zc)
    n = data.shape[1]
    x = data[0:n, :] + origin[0]
    y = data[n:2 * n, :] + origin[1]
    z = data[2 * n:, :] + origin[2]

    axes.plot_surface(x, y, z, alpha=alpha,color= color ) #cmap='hot'

def parse_log(data_file):
    df = pd.read_csv(data_file, header=[0],skipinitialspace=True) 
    df_poses = df[["pose_00", "pose_01", 'pose_02', 'pose_03','pose_10', 'pose_11', 'pose_12',
               'pose_13', 'pose_20', 'pose_21', 'pose_22', 'pose_23', 'pose_30', 'pose_31', 'pose_32', 'pose_33']]
    df_cov = df[['cov_00', 'cov_01', 'cov_02', 'cov_10', 'cov_11', 'cov_12', 'cov_20', 'cov_21', 'cov_22']]

    poses=[]
    covs=[]
    #extract poses
    for row in range(df_poses.shape[0]):
        p = np.array(df_poses.iloc[row, :]).reshape(4,4)
        poses.append(p)

    #extract covariances
    for row in range(df_cov.shape[0]):
        c = np.array(df_cov.iloc[row, :]).reshape(3,3)
        covs.append(c)
    FrameID = df['FrameID'].tolist()
    stats_df = df[['FrameID','stamp', 'Num_matches_KF','Num_matches_KF_lms','Num_matches_KF_new',
                   'Num_matches_localmap', 'Num_inliers_KF','Num_inliers_localmap','Num_triangulated',
                   'inliers_view_0', 'inliers_view_1', 'inliers_view_2', 'triangulated_view_0',
                   'triangulated_view_1', 'triangulated_view_2']]
    Num_matches_KF = df[['FrameID', 'Num_matches_KF']]
    Num_matches_KF_lms = df[['FrameID','Num_matches_KF_lms']]
    Num_matches_KF_new = df[['FrameID','Num_matches_KF_new']]
    Num_matches_localmap = df[['FrameID','Num_matches_localmap']]
    Num_inliers_KF = df[['FrameID','Num_inliers_KF']]
    Num_inliers_localmap = df[['FrameID','Num_inliers_localmap']]
    Num_triangulated = df[['FrameID','Num_triangulated']]
    return poses, covs, stats_df
    #return poses, covs, FrameID, Num_matches_KF, Num_matches_KF_lms, Num_matches_KF_new, Num_matches_localmap, Num_inliers_KF, Num_inliers_localmap,Num_triangulated



def plot_series(dfs, colName, names):
    dfs_extracted =[]
    for i in range(0, len(dfs)):
        d = dfs[i][['FrameID', colName]]
        d.rename(columns = {colName : colName+str(i)}, inplace = True)
        dfs_extracted.append(d)
    df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['FrameID'],
                                                    how='outer',sort=True), dfs_extracted)
    df_merged = df_merged.iloc[2: , :]
    df_final = df_merged.interpolate()
    cmap = plt.cm.get_cmap('turbo', 10)
    cmap=['r', 'g', 'b', 'c']
    fig = plt.figure()
    for i in range(len(dfs_extracted)):
        plt.plot(df_final['FrameID'].tolist(), df_final[colName+str(i)].tolist(), c=cmap[i])
    plt.legend(names)
    plt.show()

def plot_poses(poses_all, covs_all):
    fig = plt.figure()
    axes = fig.gca(projection='3d')
    axis_labels=["X axis", "Y axis", "Z axis"]
    axes.set_xlabel(axis_labels[0])
    axes.set_ylabel(axis_labels[1])
    axes.set_zlabel(axis_labels[2])
    cols=[np.array([1.0,0,0]),np.array([0,1.0,0])]
    i=0
    for i in range(len(poses_all)):
        #color
        poses = poses_all[i]
        covs = covs_all[i]
        col = cols[i]
        i = i+1
        print("color ", col)
        for p,c in zip(poses, covs):
            #print(p)
            #print(c)
            #print("-----------------")
            plot_pose3_on_axes(axes, p,np.array([0,1.0,0]), P=c)
    
    title="covariances"
    fig.suptitle(title)
    fig.canvas.set_window_title(title.lower())

poses_3, covs_3, stats_3 = parse_log("/home/auv/ros_ws/src/light-fields-pack/log/04_21_indoor/calib/poses_stats_isec_day_calib_5cams.txt")
poses_4, covs_4, stats_4 = parse_log("/home/auv/ros_ws/src/light-fields-pack/log/04_21_indoor/filter_intra/poses_stats_isec_day_filt_5cams.txt")
poses_5, covs_5, stats_5 = parse_log("/home/auv/ros_ws/src/light-fields-pack/log/04_25_2022_isec_day_fixed/poses_stats_isec_day_fixed_5cams.txt")

stats_3 = stats_3.iloc[4: , :]
stats_4 = stats_4.iloc[4: , :]
stats_5 = stats_5.iloc[4: , :]

stats_list = [stats_3, stats_4, stats_5]
plot_series(stats_list, 'Num_inliers_localmap', ['calib5', 'filtintra 5', 'fixed exp 5'])
plot_series(stats_list, 'inliers_view_0', ['calib5', 'filtintra 5', 'fixed exp 5'])
plot_series(stats_list, 'inliers_view_1', ['calib5', 'filtintra 5', 'fixed exp 5'])
plot_series(stats_list, 'inliers_view_2', ['calib5', 'filtintra 5', 'fixed exp 5'])
#plot_series(stats_list, 'inliers_view_3', ['calib5', '5cams', 'filtintra 5'])
#plot_series(stats_list, 'inliers_view_4', ['calib5', '5cams', 'filtintra 5'])

print("############ STATS###########")
print("Average Num of Inliers KF: {:0.2f}, {:0.2f}, {:0.2f}".format(stats_3["Num_inliers_KF"].mean(),
                                                                         stats_4["Num_inliers_KF"].mean(),
                                                                         stats_5["Num_inliers_KF"].mean()))
print("Average Num of Inliers LocalMap: {:0.2f}, {:0.2f}, {:0.2f}".format(stats_3["Num_inliers_localmap"].mean(),
                                                                         stats_4["Num_inliers_localmap"].mean(),
                                                                         stats_5["Num_inliers_localmap"].mean()))
print("Average Num of intra match inliers: {:0.2f}, {:0.2f}, {:0.2f}".format((stats_3["Num_inliers_localmap"] - stats_3["inliers_view_0"]).mean(),
                                                                      (stats_4["Num_inliers_localmap"] - stats_4["inliers_view_0"]).mean(),
                                                                      (stats_5["Num_inliers_localmap"] - stats_5["inliers_view_0"]).mean()))
print("Average Num of Mono Inliers : {:0.2f}, {:0.2f}, {:0.2f}".format(stats_3["inliers_view_0"].mean(),
                                                                         stats_4["inliers_view_0"].mean(),
                                                                         stats_5["inliers_view_0"].mean()))


print("Average Num of Triangulated points: {:0.2f}, {:0.2f}, {:0.2f}".format(stats_3["Num_triangulated"].mean(),
                                                                         stats_4["Num_triangulated"].mean(),
                                                                         stats_5["Num_triangulated"].mean()))
print("Number of keyframes: {:d}, {:d}, {:d}".format(stats_3.shape[0],
                                                          stats_4.shape[0],
                                                          stats_5.shape[0]))

#plot_poses([poses_2, poses_5], [covs_2, covs_5])

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
# for creating a responsive plot
%matplotlib widget

data_file ="../../log/02_23_2022/poses_landmark_5cams.txt"
def plt_hist_tracks(data_file, c):
    with open(data_file, 'r') as f:
        lines = f.readlines()
    flag = True
    lm_dict = {}
    hist_data = []
    kf_set = set()
    lid= -1
    for line in lines:
        vals=line.split(" ")
        if vals[0] == 'e':
            kf_set.add(vals[1])
        if vals[0] == 'l':
            if lid != -1:
                lm_dict[lid] = list(kf_set)
                hist_data.append(len(kf_set))
            lid = vals[1]
            kf_set.clear()
    print("Num Landmarks : {:d} ".format(len(lm_dict)))
    
    bins= np.arange(2, 15)
    n, x, _ = plt.hist(hist_data, bins=bins, facecolor=c, alpha=0.1, log=True)  # arguments are passed to np.histogram
    bin_centers = 0.5*(x[1:]+x[:-1])
    plt.plot(bin_centers,n, c=c)

plt_hist_tracks("../../log/02_23_2022/poses_landmark_2cams.txt",'r')
plt_hist_tracks("../../log/02_23_2022/poses_landmark_3cams.txt",'g')
plt_hist_tracks("../../log/02_23_2022/poses_landmark_4cams.txt",'b')
plt_hist_tracks("../../log/02_23_2022/poses_landmark_5cams_3.txt",'c')

plt.title("Histogram of inlier track lengths")
plt.show()

In [ ]:
import math
from scipy.spatial.transform import Rotation as R
from scipy.signal import medfilt

def cleanup_trajectory(data_file):
    roll=[]
    pit =[]
    yaw =[]
    x=[]
    poses_5, covs_5, stats_5 = parse_log(data_file)
    i=0
    for p in poses_5:
        rot = p[0:3, 0:3]
        r = R.from_matrix(rot)
        ya,pi,ro= r.as_euler('zyx', degrees=True)
        if abs(ro) > 50.0:
            if ro < 0:
                ro = ro + 180.0
            else:
                ro = 180.0-ro
        if abs(ya) > 50.0:
            if ya < 0:
                ya = ya + 180
            else:
                ya = 180.0-ya
            
        roll.append(ro)
        pit.append(pi)
        yaw.append(ya)
        x.append(i)
        i= i+1
    #write the new trajectory
    tstamps = stats_5['stamp'].tolist()
    roll_fil = medfilt(roll, 5)
    yaw_fil =  medfilt(yaw, 5)
    traj = np.zeros((len(tstamps),8))
    for i in range(len(tstamps)):
        r = R.from_euler('zyx', [yaw[i], pit[i], roll[i]], degrees=True)
        quat = r.as_quat()
        traj[i, 0] = tstamps[i]
        traj[i, 1:4] = poses_5[i][0:3, 3]
        traj[i, 4:8] = quat
    
    df_traj = pd.DataFrame(traj)
    df_traj.to_csv("../../log/02_23_2022/poses_stats_5cams_3_smooth.txt", sep=" ", index=False, header=None)
    fig = plt.figure()
    plt.plot(x,  medfilt(roll, 5), 'r-')
    plt.plot(x, pit, 'g-')
    plt.plot(x,  medfilt(yaw, 5), 'b')
    plt.show()
    
data_file = "../../log/02_23_2022/poses_stats_5cams_3.txt"
cleanup_trajectory(data_file)


In [ ]:
######Process Kimera Log
import numpy as np
import pandas as pd
import os
from datetime import datetime
import matplotlib.pyplot as plt
from matplotlib import patches
from scipy.spatial.transform import Rotation as R

data_file = "/data/results/04_21_2022/traj_vio_noshift.csv"
df = pd.read_csv(data_file, header=[0],skipinitialspace=True) 
df = df[['#timestamp','x','y','z','qx','qy','qz','qw']]
df['#timestamp'] = df['#timestamp']/1e+9

Tci = np.array([[-0.00795927, 0.99994463, -0.00688388, -0.00460387],[0.00147386, 0.00689582, 0.99997514,
                -0.01686081],[ 0.99996724, 0.00794893, -0.00152866, -0.03811018],[ 0., 0., 0., 1.]])

for row in range(df.shape[0]):
    quat = np.array(df.iloc[row, 4:8])
    rot = R.from_quat(quat).as_matrix()
    rot_1 = rot.dot(Tci[0:3,0:3].T)
    df.iloc[row, -4:] = R.from_matrix(rot_1).as_quat()
    print(row)
    print(rot)
    print("---------")
    #trans =  np.array(df.iloc[row, 1:4]).reshape(3,1)
    #tmp = Tci[0:3,0:3] @ trans
    #df.iloc[row, 1:4] = tmp.reshape(-1)
df.to_csv("/data/results/04_21_2022/Tum_trajectory_kimera_no_shift.txt", sep=" ", index=False, header=None)

In [7]:
###################################################################
#####  PROCESS the drift error from checkerboard              ##### 
###################################################################
import numpy as np
import pandas as pd
import os
from datetime import datetime
import matplotlib.pyplot as plt
from matplotlib import patches
from scipy.spatial.transform import Rotation as R
import cv2

def draw(img, corners, imgpts):
    corners = corners.astype(np.int32)
    imgpts = imgpts.astype(np.int32)
    corner = tuple(corners[0].ravel())
    img = cv2.line(img, corner, tuple(imgpts[0].ravel()), (0,0,255), 5)
    img = cv2.line(img, corner, tuple(imgpts[1].ravel()), (0,255,0), 5)
    img = cv2.line(img, corner, tuple(imgpts[2].ravel()), (255,0,0), 5)
    return img


def getPoseFromChessBoard(img_inp, k, dist, squarSize):
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
    objp = np.zeros((16*9,3), np.float32)
    objp[:,:2] = np.mgrid[0:16,0:9].T.reshape(-1,2)
    objp = objp * squarSize
    axis = np.float32([[3,0,0], [0,3,0], [0,0,-3]]).reshape(-1,3)
    axis = axis * squarSize
    T_ci = np.zeros((4,4), np.float)
    ## detect chesssboard
    ret, corners = cv2.findChessboardCorners(img_inp, (16,9),None, cv2.CALIB_CB_FAST_CHECK)
   
    if ret == True:
        print("corners found")
        corners2 = cv2.cornerSubPix(img_inp,corners,(11,11),(-1,-1),criteria)
        # Find the rotation and translation vectors.
        _,rvecs, tvecs, inliers = cv2.solvePnPRansac(objp, corners2, k, dist)
        rot_mat, _ = cv2.Rodrigues(rvecs)
        ## get pose WRT chessboard T_ci
        T_ic = np.vstack((np.hstack((rot_mat, tvecs)), np.array([0, 0, 0, 1])))
        T_ci = np.linalg.inv(T_ic)
        # project 3D points to image plane
        imgpts, jac = cv2.projectPoints(axis, rvecs, tvecs, k, dist)
        cv2.drawChessboardCorners(img_inp, (16,9), corners2, ret)
        img = cv2.cvtColor(img_inp,cv2.COLOR_GRAY2RGB)
        img = draw(img,corners2,imgpts)
        cv2.imshow('img',img)
        cv2.waitKey(0) & 0xff
    else:
        print("Corners not found")
    return T_ci
    

def computeError(df_traj, tStamp, k, dist, squareSize, patternSize, T_c0=np.eye(4)):
    img_gray = cv2.imread(os.path.join(data_path,str(tStamp)+".jpg"), 0)
    #clahe = cv2.createCLAHE(clipLimit = 5)
    #img_gray = clahe.apply(img_gray)
    T_ci_gt = getPoseFromChessBoard(img_gray, k, dist, squareSize)
    if np.linalg.norm(T_ci_gt) == 0:
        #print("Checkerboard not found")
        return -1, -1
    t =round(tStamp*1e-9, 6)
    print("Image : ", tStamp) 
    ind_df = df_traj[0].sub(t).abs().idxmin()
    
    #ind_df = (df_traj[0].tolist().index(t))
    print("index : ", ind_df)
    quat = np.array(df_traj.iloc[ind_df, 4:8])
    rot = R.from_quat(quat).as_matrix()
    trans = np.array(df_traj.iloc[ind_df,1:4]).reshape(3,1)
    T_0i = np.vstack((np.hstack((rot, trans)), np.array([0,0,0,1])))
    ## In Tum trajectory we have T_0i, pose WRT start, 
    ## GT = T_ci, expected T_ci^ = T_c0 * T_0i=
    print("GT Pose: ")
    print(T_ci_gt)
    if T_ci_gt[3,3] > 2.0 :
        print("calibrationj board is too far. estimate wont be accurate")
        return -1, -1
    print("expected Pose Traj: ")
    T_ci_exp = T_c0 @ T_0i
    print(T_ci_exp)
    #find delta pose
    deltapose = np.linalg.inv(T_ci_exp) @ T_ci_gt
    rotvec_cv,_ = cv2.Rodrigues(deltapose[0:3,0:3])
    trans_error = np.linalg.norm(T_ci_exp[0:3, 3] - T_ci_gt[0:3, 3])
    rot_error = np.rad2deg(np.linalg.norm(rotvec_cv))
    print("translation error: ",trans_error, )
    print("rotation error in degrees: ", rot_error)
    return trans_error, rot_error



# data_path="/data/04_14_2022_isec_lab_ground/isec_ground_night_2/cam0"
# traj_path = "/data/results/04_14_2022/KeyFrameTrajectory_04_14_2022_stereo.txt"
# k = np.array([[886.6272107893, 0, 361.488867356257],
#               [0, 886.7929657865, 284.068219958725],
#               [0, 0, 1]])
# dist = np.array([-0.21286163901817143, 0.17023197326272427, -9.485363152188766e-06,
#     -0.0001386285980226874])
#five_cams = [1649979950837768693,1649979951838062494, 1649979953036801584,1649979954237361874,1649979955237576739 ]
#four_cams=[1649979951437556527, 1649979952637979681,1649979953837583386, 1649979954837175857,1649979955837720923  ]
#three_cams = [1649979951838062494,1649979953036801584, 1649979954237361874,1649979955237576739 ]
#two_cams = [1649979951437556527,1649979952436942290, 1649979953637951101, 1649979954636732071, 1649979955637119932]
#orb_slam = [1649979955186799326,1649979955787700488, 1649979955987961614,1649979957087622430 ]
#five_cams1 = [1649979954037088217, 1649979955038292781, 1649979956037060130 ]
#
# first_stamp = 1649979716237968880


#data_path="/data/04_21_2022_isec_curry/bag_outdoor_nu/cam0"/data/results/04_14_2022/KeyFrameTrajectory.txt
#traj_path = "/home/auv/ros_ws/devel/lib/LFApps/04_21_2022/outdoor/Curry_center_Orbslam3_stereo.txt"
#k = np.array([[887.6164997890, 0, 361.2404351028],
#              [0, 888.007246603, 283.140828388],
#              [0, 0, 1]])
#dist = np.array([-0.21046072327790924, 0.15577979446322998, -0.0001708844706763513, -0.00022739337206347906])

#five_cams = [1650577604747234911,1650577605647659897, 1650577609847147436 ,1650577610747154366 , 1650577611947129604, 1650577613147546904]
#four_cams = [1650577604747234911, 1650577605647659897, 1650577609546509253, 1650577610447049664, 1650577611346696357, 1650577612547425164, 1650577613747880382]
#three_cams = [1650577604747234911, 1650577605647659897, 1650577609546509253, 1650577610447049664, 1650577611346696357, 1650577612547425164, 1650577613747880382]
#two_cams = [1650577604747234911,1650577605647659897, 1650577609847147436 ,1650577610747154366 , 1650577611947129604, 1650577613147546904]
#orb_slam = [ 1650577604047963739,1650577605047238159, 1650577611246613269 ]

#first_stamp = 1650576194947202033


#data_path="/data/04_21_2022_isec_curry/bag_indoor_isec/cam0"
#traj_path = "/data/results/04_21_2022/indoor/KeyFrameTrajectory.txt"
#k = np.array([[887.616499789026, 0, 361.2404351028],
#              [0, 888.007246603, 283.140828388],
#              [0, 0, 1]])
#dist = np.array([-0.21046072327790924, 0.15577979446322998, -0.0001708844706763513, -0.00022739337206347906])

#five_cams = [1650574607156144434]
#four_cams = [1650574607156144434]
#three_cams = [1650574606956391956]
#two_cams = [1650577604747234911,1650577605647659897, 1650577609847147436 ,1650577610747154366 , 1650577611947129604, 1650577613147546904]
#orb_slam = [ 1650577604047963739,1650577605047238159, 1650577611246613269 ]

#first_stamp = 1650574403456143497 #1650574407056667763 #


#data_path="/data/04_25_2022_isec_ground_dynamic/bag1/cam0"
#traj_path = "/home/auv/ros_ws/devel/lib/LFApps/04_25_2022_isec_day_fixed/04_25_2022_isec_day_fixed_Orbslam3.txt"
#k = np.array([[886.728628930516, 0, 364.89350117142],
#              [0, 887.300026814151, 286.96803692817],
#              [0, 0, 1]])
#dist = np.array([-0.21627336164628835, 0.17205009106186261, 1.633696614419175e-05, 0.0008008605016318086])
#first_stamp = 1650907022320894118 #1650907004920428538

#
data_path="/data/04_25_2022_isec_ground_dynamic/isec_ground_day_2loops/cam0"
traj_path = "/data/results/04_25_2022_4cams_traj.txt"
k = np.array([[886.7286289305, 0, 364.893501171],
             [0, 887.3000268141, 286.968036928],
            [0, 0, 1]])
dist = np.array([-0.21627336164628835, 0.17205009106186261, 1.633696614419175e-05, 0.000800860501631808])
first_stamp = 1650907610204368120
#last_stamp =  1650907894553652056        #1650907892153231019

## Get the time stamps and read images and check which ones have the checkerboard detected.
## Or get the timestamps manually and read the images

## read the tum format trajectory file
df = pd.read_csv(traj_path, header=None,skipinitialspace=True, sep=' ')
img_first = cv2.imread(os.path.join(data_path,str(first_stamp)+".jpg"), 0)
#clahe = cv2.createCLAHE(clipLimit = 5)
#img_first = clahe.apply(img_first)
T_cs = getPoseFromChessBoard(img_first, k, dist, 0.081)

# if start stamp is not the first pose
#t =round(first_stamp*1e-9, 6)
#ind_df = df[0].sub(t).abs().idxmin()
#print("index170 : ", ind_df)
#quat = np.array(df.iloc[ind_df, 4:8])
#rot = R.from_quat(quat).as_matrix()
#trans = np.array(df.iloc[ind_df,1:4]).reshape(3,1)
#T_0s = np.vstack((np.hstack((rot, trans)), np.array([0,0,0,1])))
#T_cs = T_cs @ np.linalg.inv(T_0s)  

computeError(df, first_stamp, k, dist, 0.081, (9,9), T_cs)
imgnames = os.listdir(data_path)
imgnames.sort()
imgstamps = np.array([ float(img[:-4])*1e-9 for img in imgnames])


#check_stamp = 1650574602.756657
#print("Chec stamp: ", check_stamp)
avg_trans_e = 0
avg_rot_e = 0
cnt =0
for stamp in df[0]:
    if stamp > (first_stamp*1e-9 + 170):
        print("STamp:",stamp)
        diffs = np.absolute(imgstamps - stamp)
        image_index = diffs.argmin()
        trans_e, rot_e = computeError(df, int(imgnames[image_index][:-4]), k, dist, 0.081, (9,9), T_cs)
        if trans_e != -1 and rot_e != -1:
            avg_trans_e = avg_trans_e + trans_e
            avg_rot_e = avg_rot_e + rot_e
            cnt = cnt +1
print("Average translation error : ", avg_trans_e/cnt)
print("Average rotational error : ", avg_rot_e/cnt)

def iterate_timestamp_list(stamp_list, data_path, first_stamp, k, dist):
    for last_stamp in stamp_list:
        img_first = cv2.imread(os.path.join(data_path,str(first_stamp)+".jpg"), 0)
        img_last = cv2.imread(os.path.join(data_path,str(last_stamp)+".jpg"), 0) #1650907892203072353 #1650907892153231019
        #clahe = cv2.createCLAHE(clipLimit = 5)
        #img_first = clahe.apply(img_first)
        #img_last = clahe.apply(img_last)
    
        img_combined = np.hstack((img_first, img_last))
        cv2.imshow("first and last images", img_combined)
        cv2.waitKey(0)
    
    
        T_c0 = getPoseFromChessBoard(img_first, k, dist, 0.081)
        print("First Pose WRT chessboard: ")
        print(T_c0)
    
        print("----------------------------------------------------") 
        computeError(df, first_stamp, k, dist, 0.081, (9,9), T_c0)
        computeError(df, last_stamp, k, dist, 0.081, (9,9), T_c0)
    
        cv2.destroyAllWindows()

/tmp/ipykernel_135968/3868367187.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  T_ci = np.zeros((4,4), np.float)


corners found
corners found
Image :  1650907610204368120
index :  0
GT Pose: 
[[ 8.98976328e-01 -5.61104485e-05  4.37997212e-01 -1.25970135e+00]
 [ 6.09542862e-02  9.90285097e-01 -1.24980008e-01  4.53064977e-01]
 [-4.33735099e-01  1.39051876e-01  8.90246280e-01 -3.67824729e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
expected Pose Traj: 
[[ 8.98988784e-01 -5.86699323e-05  4.37971646e-01 -1.25967111e+00]
 [ 6.09484664e-02  9.90286553e-01 -1.24971304e-01  4.53047028e-01]
 [-4.33710100e-01  1.39041500e-01  8.90260080e-01 -3.67810997e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
translation error:  0.00014174708573360352
rotation error in degrees:  0.00174262626076519
STamp: 1650907780.653292
Corners not found
STamp: 1650907781.053288
Corners not found
STamp: 1650907781.453423
Corners not found
STamp: 1650907781.652983
Corners not found
STamp: 1650907781.853002
Corners not found
STamp: 1650907782.053606
Corners not found
STamp: 1650907

Image :  1650907894553652056
index :  337
GT Pose: 
[[ 0.98603978  0.00693975  0.16636522 -0.13013057]
 [ 0.01656468  0.99008653 -0.1394786   0.41624636]
 [-0.16568391  0.14028724  0.97614975 -3.44986278]
 [ 0.          0.          0.          1.        ]]
expected Pose Traj: 
[[ 9.77305000e-01 -8.03482092e-04  2.11835530e-01  3.66602066e-01]
 [ 3.12799348e-02  9.89578368e-01 -1.40556819e-01  4.65677400e-04]
 [-2.09514923e-01  1.43993083e-01  9.67145020e-01 -3.74655890e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
translation error:  0.712492439139335
rotation error in degrees:  2.699160353765345
Average translation error :  0.6914968001475088
Average rotational error :  3.989387368682297


In [ ]:
###########################################################################
### NON-Overlapping Cams Calibration#######################################
###########################################################################
import numpy as np
import pandas as pd
import os
from datetime import datetime
import matplotlib.pyplot as plt
from matplotlib import patches
from scipy.spatial.transform import Rotation as R
import gtsam
import cv2


def draw(img, corners, imgpts):
    corners = corners.astype(np.int32)
    imgpts = imgpts.astype(np.int32)
    corner = tuple(corners[0].ravel())
    img = cv2.line(img, corner, tuple(imgpts[0].ravel()), (0,0,255), 5)
    img = cv2.line(img, corner, tuple(imgpts[1].ravel()), (0,255,0), 5)
    img = cv2.line(img, corner, tuple(imgpts[2].ravel()), (255,0,0), 5)
    return img


def getPoseFromChessBoard(img_inp, k, dist, squarSize):
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
    objp = np.zeros((16*9,3), np.float32)
    objp[:,:2] = np.mgrid[0:16,0:9].T.reshape(-1,2)
    objp = objp * squarSize
    axis = np.float32([[3,0,0], [0,3,0], [0,0,-3]]).reshape(-1,3)
    axis = axis * squarSize
    T_ci = np.zeros((4,4), np.float)
    ## detect chesssboard
    ret, corners = cv2.findChessboardCorners(img_inp, (16,9),None)
    corners2 = []
    if ret == True:
        #print("corners found")
        corners2 = cv2.cornerSubPix(img_inp,corners,(11,11),(-1,-1),criteria)
        # Find the rotation and translation vectors.
        _,rvecs, tvecs, inliers = cv2.solvePnPRansac(objp, corners2, k, dist)
        rot_mat, _ = cv2.Rodrigues(rvecs)
        ## get pose WRT chessboard T_ci
        T_ic = np.vstack((np.hstack((rot_mat, tvecs)), np.array([0, 0, 0, 1])))
        T_ci = np.linalg.inv(T_ic)
        # project 3D points to image plane
        imgpts, jac = cv2.projectPoints(axis, rvecs, tvecs, k, dist)
        cv2.drawChessboardCorners(img_inp, (16,9), corners2, ret)
        img = cv2.cvtColor(img_inp,cv2.COLOR_GRAY2RGB)
        img = draw(img,corners2,imgpts)
        cv2.imshow('img',img)
        cv2.waitKey(0) & 0xff
    return T_ci, corners2
    
def get_extrinsic_estimate(image_path, tSTamp, df, k, dist,T_opti_initial, T_board_initial):
    #Go through other camera images one by one.
    # when calibration board is detected get pose T_board_side
    #print(imgname)imgg
    img_gray = cv2.imread(image_path, 0)
    cv2.imshow("image", img_gray)
    cv2.waitKey(0)
    T_board_side = getPoseFromChessBoard(img_gray, k, dist, 0.081)
    if np.linalg.norm(T_board_side) == 0:
        #print("Checkerboard not found")
        return
    #Get the corresponding optitrack pose T_opti_rig. 
    #We want relative motion T_rig1_rig = T_rig1_opti * T_opti_rig
    print("Image : ", tStamp) 
    ind_df = df[0].sub(tStamp).abs().idxmin()
    print("index : ", ind_df)
    quat = np.array(df.iloc[ind_df, 4:8])
    rot = R.from_quat(quat).as_matrix()
    trans = np.array(df.iloc[ind_df,1:4]).reshape(3,1)
    T_opti_rig = np.vstack((np.hstack((rot, trans)), np.array([0,0,0,1])))
    T_rel_cur_1 = np.linalg.inv(T_opti_rig) @ T_opti_initial
    #compute the extrinsics T_rig_side =  T_rig_rig1 * T_rig1_board* T_board_side
    T_front_side = T_rel_cur_1 @ np.linalg.inv(T_board_initial) @ T_board_side
    print("Extrinsic T_front_side: ", T_front_side)

data_path_front="/data/05_16_2022_calib/bag3/cam0"
data_path_left="/data/05_16_2022_calib/bag3/cam5"
data_path_right="/data/05_16_2022_calib/bag3/cam6"
traj_path = "/data/05_16_2022_calib/bag3/2022_05_16_18_34_56_gt.txt"

k_f = np.array([[874.039438699, 0, 364.550740159484],
              [0, 874.787225463577, 286.6490604574],
              [0, 0, 1]])
dist_f = np.array([-0.24772900267450307, 0.1921058006879624, 2.2598774552204495e-05, 0.00309486514093092])

k_s6 = np.array([[882.9391342187388, 0, 384.1878684084419],
              [0, 882.8153341385736, 278.9079081016814],
              [0, 0, 1]])
dist_s6 = np.array([-0.21446184288970324, 0.14293621778218552, -0.0008679687005863199, 0.0013624375889589688])

k_s5 = np.array([[877.1880881359045, 0, 367.273509202746],
              [0, 877.57764402193, 285.9607389654299],
              [0, 0, 1]])
dist_s5 = np.array([-0.2228949253370626, 0.18736719172646038, 0.0010781783131958086, -0.0005436650475248478])

first_stamp =  1652740514096921457 #side6 1652740503496921457

#read the optitrack trajectory file
df = pd.read_csv(traj_path, header=None,skipinitialspace=True, sep=' ')

#get the pose of the camera WRT calib board . This is initial pose. T_board_rig1
img_first = cv2.imread(os.path.join(data_path_front,str(first_stamp)+".png"), 0)
T_board_front1, _ = getPoseFromChessBoard(img_first, k_f, dist_f, 0.081)
print(T_board_front1)

#get the corresponding optitrack pose. T_opti_rig1
t =round(first_stamp*1e-9, 6)
print("Image : ", first_stamp) 
ind_df = df[0].sub(t).abs().idxmin()
print("index : ", ind_df)
quat = np.array(df.iloc[ind_df, 4:8])
rot = R.from_quat(quat).as_matrix()
trans = np.array(df.iloc[ind_df,1:4]).reshape(3,1)
T_opti_rig1 = np.vstack((np.hstack((rot, trans)), np.array([0,0,0,1])))

imgnames = os.listdir(data_path_left)
imgnames.sort()
imgstamps = np.array([ float(img[:-4])*1e-9 for img in imgnames])

#for imgname, tStamp in zip(imgnames[312 :412], imgstamps[312:412]):
#    #get the pose of the camera WRT calib board . This is initial pose. T_board_rig1
#    img_first = cv2.imread(os.path.join(data_path_front,imgname), 0)
#    T_board_1 = getPoseFromChessBoard(img_first, k_f, dist_f, 0.081)
#    
#    #get the corresponding optitrack pose. T_opti_rig1
#    print("Image : ", tStamp) 
#    ind_df = df[0].sub(tStamp).abs().idxmin()
#    print("index : ", ind_df)
#    quat = np.array(df.iloc[ind_df, 4:8])
#    rot = R.from_quat(quat).as_matrix()
#    trans = np.array(df.iloc[ind_df,1:4]).reshape(3,1)
#    T_opti_1 = np.vstack((np.hstack((rot, trans)), np.array([0,0,0,1])))
#    T_opti_board = T_opti_1 @ np.linalg.inv(T_board_1)
#    print("T_opti_Board: data_path_right",T_opti_board )
#    #get_extrinsic_estimate(os.path.join(data_path_front,imgname),tStamp, df, k_f, dist_f, T_opti_rig1, T_board_front1)
#   
#for imgname, tStamp in zip(imgnames[:100], imgstamps[:100]):
#    get_extrinsic_estimate(os.path.join(data_path_right,imgname),tStamp, df, k_s6, dist_s6, T_opti_rig1, T_board_front1)
#
#for imgname, tStamp in zip(imgnames[400:500], imgstamps[400:500]):
#    get_extrinsic_estimate(os.path.join(data_path_left,imgname),tStamp, df, k_s5, dist_s5, T_opti_rig1, T_board_front1)

#############################
#### Optimization ###########
#############################

#We have initial estimates for landmarks(calibration target points)
#rig poses frok optitrack and the relative poses between the rig and the individual cameras
T_front_left = np.array([[-0.00728608, -0.02427516, -0.99967876 , 0.27547358],
 [ 0.02150753,  0.99947024, -0.02442685, -0.13939047],
 [ 0.99974214, -0.0216786,  -0.00676012, -0.03593664],
 [ 0. ,         0. ,         0.   ,       1.        ]])

T_front_right = np.array( [[ 0.00935536,  0.01434981,  0.99985327 , 0.37059141],
 [-0.01839862,  0.99973023, -0.01417589, -0.13330094],
 [-0.99978696, -0.0182633,   0.00961685 , 0.00726508],
 [ 0.,          0.,          0.,          1.        ]])


lms_board = np.zeros((16*9,3), np.float32)
lms_board[:,:2] = np.mgrid[0:16,0:9].T.reshape(-1,2)
lms_board = lms_board * 0.081
lms_board = np.hstack((lms_board, np.ones((16*9, 1))))

#print(lms_board)
lms_1 = np.linalg.inv(T_board_front1) @ lms_board.T  

# We build a factor graph with projectfactorPPP which optimizes over point, pose and pose
# Read the front camera images. For first pose get T_first_board. This pose is the origin. We also have T_opti_rig1

# for each of the other images, get the VO from optitrack T_rig1_opti * T_opti_rig = T_1_cur
# insert the projection factor where the lms are WRT first cam = T_first_board * lms_board, 
# pose of the rig which coincides with first camera T_1_cur (this is indentity for the first frame)
# pose of the component camera WRT first camera T_cur_left, T_cur_right
from gtsam import symbol_shorthand
L = symbol_shorthand.L
X = symbol_shorthand.X
B = symbol_shorthand.B
import gtsam_unstable
from gtsam_unstable import ProjectionFactorPPPCal3_S2
from gtsam import (Cal3_S2, LevenbergMarquardtOptimizer,
                         GenericProjectionFactorCal3_S2, Marginals,
                         NonlinearFactorGraph, PinholeCameraCal3_S2, Point3, Point2,
                         Pose3, PriorFactorPoint3, PriorFactorPose3, Rot3, Values)


gtsam_Kf = Cal3_S2(k_f[0,0], k_f[1,1], 0.0, k_f[0,2], k_f[1,2])
gtsam_Kleft = Cal3_S2(k_s5[0,0], k_s5[1,1], 0.0, k_s5[0,2], k_s5[1,2])
gtsam_Kright = Cal3_S2(k_s6[0,0], k_s6[1,1], 0.0, k_s6[0,2], k_s6[1,2])


# Define the camera observation noise model
measurement_noise = gtsam.noiseModel.Isotropic.Sigma(2, 2.0)  # one pixel in u and v
#0.3 rad and 0.1 m
pose_noise = gtsam.noiseModel.Diagonal.Sigmas(np.array([0.1, 0.1, 0.1, 0.05, 0.05, 0.05]))
# 0.1 m
point_noise = gtsam.noiseModel.Isotropic.Sigma(3, 0.1)

#Initialze graph and initial values
graph = NonlinearFactorGraph()
initial_estimate = Values()

img_list = list(zip(imgnames, imgstamps))
img_index = 0

# For each image
for imgname, tStamp in img_list[0:600:10]:
    print(imgname, tStamp)
    img_index = img_index + 1
    # read the image
    img_first = cv2.imread(os.path.join(data_path_front,imgname), 0)
    #undistort the image     
    imgg = cv2.undistort(img_first, k_f, dist_f, None, k_f)
    # get the pose of the image WRT board and the corresponding image points
    T_board_front, imgPoints = getPoseFromChessBoard(imgg, k_f, np.zeros((1,4)), 0.081)
    cam_ind = 0
    # if no board detected in the fornt camera go to left ca,
    if np.linalg.norm(T_board_front) == 0:
        img_left = cv2.imread(os.path.join(data_path_left,imgname), 0)
        imgg = cv2.undistort(img_left,k_s5, dist_s5, None, k_s5)
        T_board_left, imgPoints = getPoseFromChessBoard(imgg, k_s5, np.zeros((1,4)), 0.081)
        # if no board detected in left camera go to right
        if np.linalg.norm(T_board_left) == 0:
            img_right = cv2.imread(os.path.join(data_path_right,imgname), 0)
            imgg = cv2.undistort(img_right,k_s6, dist_s6, None, k_s6)
            T_board_right, imgPoints = getPoseFromChessBoard(imgg, k_s6, np.zeros((1,4)), 0.081)
            if np.linalg.norm(T_board_right) == 0:
                print("Checkerboard not found in any of the images")
                cam_ind = -1
                continue
            else:
                cam_ind = 2
        else:
            cam_ind = 1
    if len(imgPoints) != 0:
        cv2.drawChessboardCorners(imgg, (16,9), imgPoints, True)
        cv2.imshow('img',imgg)
        cv2.waitKey(0) & 0xff
    else:
        continue
    
        
    #get the corresponding optitrack pose. T_opti_rig1
    print("Image : ", tStamp) 
    ind_df = df[0].sub(tStamp).abs().idxmin()
    print("index : ", ind_df)
    quat = np.array(df.iloc[ind_df, 4:8])
    rot = R.from_quat(quat).as_matrix()
    trans = np.array(df.iloc[ind_df,1:4]).reshape(3,1)
    T_opti_rig = np.vstack((np.hstack((rot, trans)), np.array([0,0,0,1])))
    # get the relative pose of the current frame WRT the refrence frame . 
    T_1_cur = np.linalg.inv(T_opti_rig1) @ T_opti_rig
    #  depending on the camera it is seen in get the 3d-2d correspondences
    #For each landmark add a prior if needed
    assert (len(imgPoints) == lms_1.shape[1])
    
    #Insert the pose of the rig/front camera WRT the starting position
    initial_estimate.insert(X(img_index), Pose3(T_1_cur))
    #factor = PriorFactorPose3(X(img_index), Pose3(T_1_cur), pose_noise)
    #graph.push_back(factor)
    
    K = gtsam_Kf
    if cam_ind == 1 :
        K = gtsam_Kleft
    elif cam_ind == 2:
        K = gtsam_Kright
    # Add the projectionPPP factor for each landmark with the measured corners
    for i in range(lms_1.shape[1]):
        lm = Point3(lms_1[0:3, i])
        #factor = PriorFactorPoint3(L(i), lm, point_noise)
        #graph.push_back(factor)
        meas = Point2(imgPoints[i].ravel())
        factor = ProjectionFactorPPPCal3_S2(meas, measurement_noise, X(img_index),B(cam_ind), L(i), K )
        graph.push_back(factor)

# add prior on the first pose and the front camera ad identity. This our refernce frame. We assume
# here that the front camera frame is coinciding with the first image frame
pose_noise2 = gtsam.noiseModel.Diagonal.Sigmas(np.array([0.1, 0.1, 0.1, 0.001, 0.001, 0.001]))
factor = PriorFactorPose3(B(0),Pose3(np.eye(4)), pose_noise2)
graph.push_back(factor)
initiPose = Pose3(np.eye(4))
factor = PriorFactorPose3(X(354), initiPose, pose_noise)
graph.push_back(factor)
#factor = PriorFactorPoint3(L(0), Point3(lms_1[0:3, 0]), point_noise)
#graph.push_back(factor)

############################
## Initial Estimates########
############################
initial_estimate.insert(X(354), Pose3(np.eye(4)))
initial_estimate.insert(B(0), Pose3(np.eye(4)))
initial_estimate.insert(B(1), Pose3(T_front_left))
initial_estimate.insert(B(2), Pose3(T_front_right))
for i in range(lms_1.shape[1]):
    lm = Point3(lms_1[0:3, i])
    initial_estimate.insert(L(i), lm)


params = gtsam.LevenbergMarquardtParams()
params.setDiagonalDamping(True)
params.setVerbosityLM("SUMMARY")
optimizer = LevenbergMarquardtOptimizer(graph, initial_estimate, params)
result = optimizer.optimize()
result.print('Final results:\n')
print('initial error = {}'.format(graph.error(initial_estimate)))
print('final error = {}'.format(graph.error(result)))


In [ ]:
###########################################################################
### NON-Overlapping Cams Calibration#######################################
###########################################################################
import numpy as np
import pandas as pd
import os
from datetime import datetime
import matplotlib.pyplot as plt
from matplotlib import patches
from scipy.spatial.transform import Rotation as R
import gtsam
import cv2


def draw(img, corners, imgpts):
    corners = corners.astype(np.int32)
    imgpts = imgpts.astype(np.int32)
    corner = tuple(corners[0].ravel())
    img = cv2.line(img, corner, tuple(imgpts[0].ravel()), (0,0,255), 5)
    img = cv2.line(img, corner, tuple(imgpts[1].ravel()), (0,255,0), 5)
    img = cv2.line(img, corner, tuple(imgpts[2].ravel()), (255,0,0), 5)
    return img


def getPoseFromChessBoard(img_inp, k, dist, squarSize):
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
    objp = np.zeros((16*9,3), np.float32)
    objp[:,:2] = np.mgrid[0:16,0:9].T.reshape(-1,2)
    objp = objp * squarSize
    axis = np.float32([[3,0,0], [0,3,0], [0,0,-3]]).reshape(-1,3)
    axis = axis * squarSize
    T_ci = np.zeros((4,4), np.float)
    ## detect chesssboard
    ret, corners = cv2.findChessboardCorners(img_inp, (16,9),None)
    corners2 = []
    if ret == True:
        #print("corners found")
        corners2 = cv2.cornerSubPix(img_inp,corners,(11,11),(-1,-1),criteria)
        # Find the rotation and translation vectors.
        _,rvecs, tvecs, inliers = cv2.solvePnPRansac(objp, corners2, k, dist)
        rot_mat, _ = cv2.Rodrigues(rvecs)
        ## get pose WRT chessboard T_ci
        T_ic = np.vstack((np.hstack((rot_mat, tvecs)), np.array([0, 0, 0, 1])))
        T_ci = np.linalg.inv(T_ic)
        # project 3D points to image plane
        imgpts, jac = cv2.projectPoints(axis, rvecs, tvecs, k, dist)
        cv2.drawChessboardCorners(img_inp, (16,9), corners2, ret)
        img = cv2.cvtColor(img_inp,cv2.COLOR_GRAY2RGB)
        img = draw(img,corners2,imgpts)
        cv2.imshow('img',img)
        cv2.waitKey(0) & 0xff
    return T_ci, corners2
##########################################################################
######## These are paths and camera parameters I used, just for an example
#################################################################################
data_path_front="/data/05_16_2022_calib/bag3/cam0"
data_path_left="/data/05_16_2022_calib/bag3/cam5"
data_path_right="/data/05_16_2022_calib/bag3/cam6"
traj_path = "/data/05_16_2022_calib/bag3/2022_05_16_18_34_56_gt.txt"

k_f = np.array([[874.039438699, 0, 364.550740159484],
              [0, 874.787225463577, 286.6490604574],
              [0, 0, 1]])
dist_f = np.array([-0.24772900267450307, 0.1921058006879624, 2.2598774552204495e-05, 0.00309486514093092])

k_s6 = np.array([[882.9391342187388, 0, 384.1878684084419],
              [0, 882.8153341385736, 278.9079081016814],
              [0, 0, 1]])
dist_s6 = np.array([-0.21446184288970324, 0.14293621778218552, -0.0008679687005863199, 0.0013624375889589688])

k_s5 = np.array([[877.1880881359045, 0, 367.273509202746],
              [0, 877.57764402193, 285.9607389654299],
              [0, 0, 1]])
dist_s5 = np.array([-0.2228949253370626, 0.18736719172646038, 0.0010781783131958086, -0.0005436650475248478])

first_stamp =  1652740514096921457 

############################################################################


#read the optitrack trajectory file into a pandas dataframe
# the trajectory file is in TUM format and contains poses of the multi-camera rig
# format -> timestamp tx ty tz qx qy qz qw
df = pd.read_csv(traj_path, header=None,skipinitialspace=True, sep=' ')

#Our multi-camera rig has 3 non-overlapping cameras front, left and right.
#front camera is considered as the reference coordinate frame for the rig.
#Read the first image in front camera in the sequence. 
#get the pose of the front camera WRT calib board for the first frame in the sequence. 
#his pose is the origin, initial pose of the rig.
img_first = cv2.imread(os.path.join(data_path_front,str(first_stamp)+".png"), 0)
T_board_front1, _ = getPoseFromChessBoard(img_first, k_f, dist_f, 0.081)
print(T_board_front1)

#get the corresponding optitrack pose. T_opti_front1
t =round(first_stamp*1e-9, 6)
print("Image : ", first_stamp) 
ind_df = df[0].sub(t).abs().idxmin()
print("index : ", ind_df)
quat = np.array(df.iloc[ind_df, 4:8])
rot = R.from_quat(quat).as_matrix()
trans = np.array(df.iloc[ind_df,1:4]).reshape(3,1)
T_opti_front1 = np.vstack((np.hstack((rot, trans)), np.array([0,0,0,1])))

imgnames = os.listdir(data_path_left)
imgnames.sort()
imgstamps = np.array([ float(img[:-4])*1e-9 for img in imgnames])

############################################
######## Pose graph Optimization ###########
############################################

#We have initial estimates for landmarks(calibration target points)
#rig poses from optitrack and the relative poses between the rig and the individual cameras
# Front camera is assumed to be the reference frame for the multi-camera rig.

T_front_left = np.array([[-0.00728608, -0.02427516, -0.99967876 , 0.27547358],
 [ 0.02150753,  0.99947024, -0.02442685, -0.13939047],
 [ 0.99974214, -0.0216786,  -0.00676012, -0.03593664],
 [ 0. ,         0. ,         0.   ,       1.        ]])

T_front_right = np.array( [[ 0.00935536,  0.01434981,  0.99985327 , 0.37059141],
 [-0.01839862,  0.99973023, -0.01417589, -0.13330094],
 [-0.99978696, -0.0182633,   0.00961685 , 0.00726508],
 [ 0.,          0.,          0.,          1.        ]])

### landmarks are the chessboard corners
lms_board = np.zeros((16*9,3), np.float32)
lms_board[:,:2] = np.mgrid[0:16,0:9].T.reshape(-1,2)
lms_board = lms_board * 0.081
lms_board = np.hstack((lms_board, np.ones((16*9, 1))))

#Convert the landmarks from the board coordinate frame to 
# multi-camera rig coordinate frame (front camera coordinate frame)
lms_1 = np.linalg.inv(T_board_front1) @ lms_board.T  

# We build a factor graph with projectfactorPPP which optimizes over point, pose (rig) and pose (component camera)
# insert the projection factor where the lms are WRT first cam = T_first_board * lms_board, 
# pose of the rig which coincides with first camera T_1_cur (this is indentity for the first frame)
# pose of the component camera WRT first camera T_cur_left, T_cur_right
from gtsam import symbol_shorthand
L = symbol_shorthand.L
X = symbol_shorthand.X
B = symbol_shorthand.B
import gtsam_unstable
from gtsam_unstable import ProjectionFactorPPPCal3_S2
from gtsam import (Cal3_S2, LevenbergMarquardtOptimizer,
                         GenericProjectionFactorCal3_S2, Marginals,
                         NonlinearFactorGraph, PinholeCameraCal3_S2, Point3, Point2,
                         Pose3, PriorFactorPoint3, PriorFactorPose3, Rot3, Values)


gtsam_Kf = Cal3_S2(k_f[0,0], k_f[1,1], 0.0, k_f[0,2], k_f[1,2])
gtsam_Kleft = Cal3_S2(k_s5[0,0], k_s5[1,1], 0.0, k_s5[0,2], k_s5[1,2])
gtsam_Kright = Cal3_S2(k_s6[0,0], k_s6[1,1], 0.0, k_s6[0,2], k_s6[1,2])


# Define the camera observation noise model
measurement_noise = gtsam.noiseModel.Isotropic.Sigma(2, 2.0)  # one pixel in u and v
#0.3 rad and 0.1 m
pose_noise = gtsam.noiseModel.Diagonal.Sigmas(np.array([0.1, 0.1, 0.1, 0.05, 0.05, 0.05]))
# 0.1 m
point_noise = gtsam.noiseModel.Isotropic.Sigma(3, 0.1)

#Initialze graph and initial values
graph = NonlinearFactorGraph()
initial_estimate = Values()

img_list = list(zip(imgnames, imgstamps))
img_index = 0

# For each image
for imgname, tStamp in img_list[0:600:10]:
    print(imgname, tStamp)
    img_index = img_index + 1
    # read the image in the front camera
    img_first = cv2.imread(os.path.join(data_path_front,imgname), 0)
    #undistort the image     
    imgg = cv2.undistort(img_first, k_f, dist_f, None, k_f)
    # get the pose of the image WRT board and the corresponding image points
    T_board_front, imgPoints = getPoseFromChessBoard(imgg, k_f, np.zeros((1,4)), 0.081)
    cam_ind = 0
    # if no board detected in the fornt camera go to left camera
    if np.linalg.norm(T_board_front) == 0:
        img_left = cv2.imread(os.path.join(data_path_left,imgname), 0)
        imgg = cv2.undistort(img_left,k_s5, dist_s5, None, k_s5)
        T_board_left, imgPoints = getPoseFromChessBoard(imgg, k_s5, np.zeros((1,4)), 0.081)
        # if no board detected in left camera go to right
        if np.linalg.norm(T_board_left) == 0:
            img_right = cv2.imread(os.path.join(data_path_right,imgname), 0)
            imgg = cv2.undistort(img_right,k_s6, dist_s6, None, k_s6)
            T_board_right, imgPoints = getPoseFromChessBoard(imgg, k_s6, np.zeros((1,4)), 0.081)
            if np.linalg.norm(T_board_right) == 0:
                print("Checkerboard not found in any of the images")
                cam_ind = -1
                continue
            else:
                cam_ind = 2
        else:
            cam_ind = 1
    # if no calibration target is detected in any of the cameras go to next
    if len(imgPoints) != 0:
        cv2.drawChessboardCorners(imgg, (16,9), imgPoints, True)
        cv2.imshow('img',imgg)
        cv2.waitKey(0) & 0xff
    else:
        continue
    
        
    #get the corresponding optitrack pose. T_opti_rig1
    print("Image : ", tStamp) 
    ind_df = df[0].sub(tStamp).abs().idxmin()
    print("index : ", ind_df)
    quat = np.array(df.iloc[ind_df, 4:8])
    rot = R.from_quat(quat).as_matrix()
    trans = np.array(df.iloc[ind_df,1:4]).reshape(3,1)
    T_opti_cur = np.vstack((np.hstack((rot, trans)), np.array([0,0,0,1])))
    
    # get the relative pose of the current frame WRT the initial frame(origin) . 
    T_1_cur = np.linalg.inv(T_opti_front1) @ T_opti_cur
    #  depending on the camera it is seen in get the 3d-2d correspondences
    #For each landmark add a prior if needed
    assert (len(imgPoints) == lms_1.shape[1])
    
    #Insert the pose of the rig/front camera WRT the starting position
    initial_estimate.insert(X(img_index), Pose3(T_1_cur))
    #factor = PriorFactorPose3(X(img_index), Pose3(T_1_cur), pose_noise)
    #graph.push_back(factor)
    
    K = gtsam_Kf
    if cam_ind == 1 :
        K = gtsam_Kleft
    elif cam_ind == 2:
        K = gtsam_Kright
    # Add the projectionPPP factor for each landmark with the measured corners
    for i in range(lms_1.shape[1]):
        lm = Point3(lms_1[0:3, i])
        #factor = PriorFactorPoint3(L(i), lm, point_noise)
        #graph.push_back(factor)
        meas = Point2(imgPoints[i].ravel())
        factor = ProjectionFactorPPPCal3_S2(meas, measurement_noise, X(img_index),B(cam_ind), L(i), K )
        graph.push_back(factor)

# add prior on the first pose and the front camera as identity. This our refernce frame. 
pose_noise2 = gtsam.noiseModel.Diagonal.Sigmas(np.array([0.1, 0.1, 0.1, 0.001, 0.001, 0.001]))
factor = PriorFactorPose3(B(0),Pose3(np.eye(4)), pose_noise2)
graph.push_back(factor)
initiPose = Pose3(np.eye(4))
factor = PriorFactorPose3(X(354), initiPose, pose_noise)
graph.push_back(factor)
#factor = PriorFactorPoint3(L(0), Point3(lms_1[0:3, 0]), point_noise)
#graph.push_back(factor)

############################
## Initial Estimates########
############################
initial_estimate.insert(X(354), Pose3(np.eye(4)))
initial_estimate.insert(B(0), Pose3(np.eye(4)))
initial_estimate.insert(B(1), Pose3(T_front_left))
initial_estimate.insert(B(2), Pose3(T_front_right))
for i in range(lms_1.shape[1]):
    lm = Point3(lms_1[0:3, i])
    initial_estimate.insert(L(i), lm)


params = gtsam.LevenbergMarquardtParams()
params.setDiagonalDamping(True)
params.setVerbosityLM("SUMMARY")
optimizer = LevenbergMarquardtOptimizer(graph, initial_estimate, params)
result = optimizer.optimize()
result.print('Final results:\n')
print('initial error = {}'.format(graph.error(initial_estimate)))
print('final error = {}'.format(graph.error(result)))

In [ ]:
np.linalg.inv(T_front_right)

In [ ]:
###########################################################################
### PROCESS Groundtruth Data -GPS ############################################
############################################################################
import glob
import pickle
import utm
2 - 0.460
3 - 0.41
4 - 0.393
5 - 0.361
#read the pkl file of gps
data_path = "/data/05_05_2022_whoi/fixed_expo"
files = glob.glob(os.path.join(data_path, "*.pkl")) #os.listdir(os.path.join(data_path)
files.sort()
print(files[0])
all_data=[]
for f in files:
    df = pd.read_pickle(f)
    df_gps = df['robot']['gps']
    df_gps.reset_index(inplace=True)
    print("Read ", f)
    for i in range(df_gps.shape[0]):
        x, y, _,_ = utm.from_latlon(df_gps.iloc[i,1], df_gps.iloc[i,2])
        z = df_gps.iloc[i,3]
        t = pd.Timestamp(df_gps.iloc[i,0]).timestamp()
        all_data.append([t, -1.0*y, z, x, 0.0, 0.0, 0.0, 1.0])
        #print(t," ",x," ",y," ",z)
np_data = np.array(all_data, np.float64)
pd_data = pd.DataFrame(np_data)
print(pd_data.head())
print(pd_data.tail())
pd_data[[1,2,3]] = pd_data[[1,2,3]] - pd_data.iloc[0, 1:4]
print(pd_data.head())
pd_data.to_csv(os.path.join(data_path, "gps_groundtruth.txt"), sep=' ', header=False, index=False)
print(pd_data.tail())

In [ ]:
./LFSlamapp --config_file /data/05_15_2022/bag2/lf_slam_config.cfg --log_file /data/05_15_2022/bag2/results/pose_stats_05_15_2022_5cams_1.txt --traj_file /data/05_15_2022/bag2/results/traj_05_15_2022_5cams_1.txt -v 1
./LFSlamapp --config_file /data/05_15_2022/bag2/lf_slam_config.cfg --log_file /data/05_15_2022/bag2/results/pose_stats_05_15_2022_5cams_2.txt --traj_file /data/05_15_2022/bag2/results/traj_05_15_2022_5cams_2.txt -v 1
./LFSlamapp --config_file /data/05_15_2022/bag2/lf_slam_config.cfg --log_file /data/05_15_2022/bag2/results/pose_stats_05_15_2022_5cams_3.txt --traj_file /data/05_15_2022/bag2/results/traj_05_15_2022_5cams_3.txt -v 1